# Учебный проект 6_Линейные модели машинного обучения

## Содержание

* [Описание проекта](#описание-проекта)
* [Импорт библиотек Python и загрузка данных](#импорт-библиотек-python-и-загрузка-данных)
* [Предобработка данных](#предобработка-данных)
* [Исследовательский анализ данных](#исследовательский-анализ-данных)
* [Объединение данных](#объединение-данных)

## Описание проекта <a class = 'anchor' id = 'описание'></a>

На исследовании находятся данные `о текущем поголовье скота, родословной каждой коровы и список коров для будущей покупки`, которые были предоставлены фермером.

---

`Задача`

**Разработать модели машинного обучения для принятия решений о покупке коров на основании показателей**:

1. Средний удой коровы - **не менее 6 000 килограммов**;
2. **Молоко коров должно быть вкусным**.

---

`Путь решения`

Подготовить модели машинного обучения:
* Для прогнозирования удоя коровы - **модель линейной регрессии** (так как размер удоя - количественный показатель);
* Для прогнозирования вкуса молока - **модель логистической регрессии** (вкус молока - качественный показатель).

---

`Располагаемые данные`

**Данные о текущем состоянии стада - ferma_main**

* id - уникальный идентификатор коровы;
* Удой, кг — масса молока, которую корова даёт в год (в килограммах);
* ЭКЕ (Энергетическая кормовая единица) — показатель питательности корма коровы;
* Сырой протеин, г — содержание сырого протеина в корме (в граммах);
* СПО (Сахаро-протеиновое соотношение) — отношение сахара к протеину в корме коровы;
* Порода — порода коровы;
* Тип пастбища — ландшафт лугов, на которых паслась корова;
*порода папы_быка — порода папы коровы;
* Жирность,% — содержание жиров в молоке (в процентах);
* Белок,% — содержание белков в молоке (в процентах);
* Вкус молока — оценка вкуса по личным критериям фермера, бинарный признак (вкусно, не вкусно);
* Возраст — возраст коровы, бинарный признак (менее_2_лет, более_2_лет).

**Данные о родословной каждой коровы - ferma_dad**

* id — уникальный идентификатор коровы.;
* Имя Папы — имя папы коровы.

**Данные о списке коров для покупки - cow_buy**

* Порода — порода коровы;
* Тип пастбища — ландшафт лугов, на которых паслась корова;
* порода папы_быка — порода папы коровы;
* Имя_папы — имя папы коровы;
* Текущая_жирность,% — содержание жиров в молоке (в процентах);
* Текущий_уровень_белок,% — содержание белков в молоке (в процентах);
* Возраст — возраст коровы, бинарный признак (менее_2_лет, более_2_лет).

## Импорт библиотек Python и загрузка данных <a class = anchor id = 'Импорт'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Импорт библиотек Python:
    * для манипулирования данными;
    * для визуализации данных;
    * для решения задач машинного обучения.
2. Загрузка данных в рабочую среду Jupyter Notebook. Инициализация переменных в соответствие с названиями загружаемых датасетов: **ferma_main**, **ferma_dad**, **cow_buy**;
3. Вывод на экран параметров датасетов:
    * вывод общей структуры набора данных - демонстрация первых 5 строк;
    * общей информации о наборе данных;
4. Формирование вывода по итогам данного этапа.

In [1]:
# импорт библиотек python

# для манипулирования данными
import pandas as pd
import numpy as np

# для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# методы машинного обучения
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

In [7]:
# загрузка данных в рабочую среду
try:
    ferma_main = pd.read_csv('~/Desktop/YandexPraktikum_projects/datasets/ferma_main.csv', sep = ';') # основной набор данных о текущем состоянии стада
    ferma_dad = pd.read_csv('~/Desktop/YandexPraktikum_projects/datasets/ferma_dad.csv', sep = ';') # данные о родословной каждой коровы
    cow_buy = pd.read_csv('~/Desktop/YandexPraktikum_projects/datasets/cow_buy.csv', sep = ';') # данные о списке коров для покупки
except:
    ferma_main = pd.read_csv('/datasets/ferma_main.csv', sep = ';')
    ferma_dad = pd.read_csv('/datasets/ferma_dad.csv', sep = ';')
    cow_buy = pd.read_csv('/datasets/cow_buy.csv', sep = ';')

In [8]:
# пользовательская функция для первичного изучения содержимого наборов данных
def  first_meeting (df : pd.DataFrame, df_name : str) -> None:
    print(f'Структура набора данных {df_name}')
    display(df.head())
    print()
    print(df.info())

# вывод на экран параметров датасетов
first_meeting(ferma_main, 'ferma_main')
first_meeting(ferma_dad, 'ferma_dad')
first_meeting(cow_buy, 'cow_buy')

Структура набора данных ferma_main


,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст
0,1,5863,"14,2",1743,"0,89",Вис Бик Айдиал,Равнинное,Айдиал,"3,58","3,076",вкусно,более_2_лет
1,2,5529,"12,8",2138,"0,89",Вис Бик Айдиал,Равнинные,Соверин,"3,54","3,079",вкусно,менее_2_лет
2,3,5810,14,1854,"0,885",РефлешнСоверинг,Холмистое,Соверин,"3,59","3,074",не вкусно,более_2_лет
3,4,5895,"12,4",2012,"0,885",РефлешнСоверинг,Холмистое,Айдиал,"3,4","3,075",не вкусно,более_2_лет
4,5,5302,"12,8",1675,"0,885",Вис Бик Айдиал,Равнинные,Соверин,"3,73","3,073",вкусно,менее_2_лет



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   id                                     634 non-null    int64 
 1   Удой, кг                               634 non-null    int64 
 2   ЭКЕ (Энергетическая кормовая единица)  634 non-null    object
 3   Сырой протеин, г                       634 non-null    int64 
 4   СПО (Сахаро-протеиновое соотношение)   634 non-null    object
 5   Порода                                 634 non-null    object
 6   Тип пастбища                           634 non-null    object
 7   порода папы_быка                       634 non-null    object
 8   Жирность,%                             634 non-null    object
 9   Белок,%                                634 non-null    object
 10  Вкус молока                            634 non-null    object
 11  Возраст           

,id,Имя Папы
0,1,Буйный
1,2,Соловчик
2,3,Барин
3,4,Буйный
4,5,Барин



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        629 non-null    int64 
 1   Имя Папы  629 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.0+ KB
None
Структура набора данных cow_buy


,Порода,Тип пастбища,порода папы_быка,Имя_папы,"Текущая_жирность,%","Текущий_уровень_белок,%",Возраст
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,"3,58","3,076",более_2_лет
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,"3,54","3,081",менее_2_лет
2,РефлешнСоверинг,равнинное,Соверин,Барин,"3,59","3,074",более_2_лет
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,"3,4","3,061",более_2_лет
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,"3,64","3,074",более_2_лет



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Порода                   20 non-null     object
 1   Тип пастбища             20 non-null     object
 2   порода папы_быка         20 non-null     object
 3   Имя_папы                 20 non-null     object
 4   Текущая_жирность,%       20 non-null     object
 5   Текущий_уровень_белок,%  20 non-null     object
 6   Возраст                  20 non-null     object
dtypes: object(7)
memory usage: 1.2+ KB
None


In [9]:
# вывод на экран статистической информации по наборам данных

print('Статистическая информация по набору ferma_main')
display(ferma_main.describe(include = 'all'))
print()
print('Статистическая информация по набору cow_buy')
display(cow_buy.describe(include = 'all'))

Статистическая информация по набору users


,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст
count,634.000000,634.000000,634,634.000000,634,634,634,634,634,634,634,634
unique,NaN,NaN,55,NaN,12,2,3,3,50,15,2,2
top,NaN,NaN,"14,7",NaN,"0,94",Вис Бик Айдиал,Холмистое,Соверин,"3,72","3,076",вкусно,более_2_лет
freq,NaN,NaN,27,NaN,172,373,356,316,112,103,371,524
mean,317.460568,6187.025237,NaN,1922.682965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,183.096982,1638.401021,NaN,182.392548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,5204.000000,NaN,1660.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,159.250000,5751.750000,NaN,1772.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,317.500000,6130.500000,NaN,1880.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,475.750000,6501.000000,NaN,2058.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Статистическая информация по набору rides


,Порода,Тип пастбища,порода папы_быка,Имя_папы,"Текущая_жирность,%","Текущий_уровень_белок,%",Возраст
count,20,20,20,20,20,20,20
unique,2,2,2,4,12,12,2
top,РефлешнСоверинг,равнинное,Соверин,Буйный,"3,59","3,074",более_2_лет
freq,11,11,12,8,3,5,15


**Вывод**:
1. Импортированы библиотеки Python:
    * для манипулирования данными - **pandas, numpy**;
    * для визуализации данных - **matplotlib, seaborn**;
    * для решения задач машинного обучения - **sklearn**.
2. Проведена загрузка данных в рабочую среду Jupyter Notebook. Инициализированы переменные: `ferma_main`, `ferma_dad`, `cow_buy`;
3. Выведены на экран параметры датасетов:
    * общая структура наборов данных - демонстрация первых 5 строк;
    * общая информация о наборах данных;

**Вывод после первичного знакомства с данными**:

`ferma_main`
* **В таблице нет пропущенных значений** - вся информация о поголовье коров и их характеристиках доступна;
* Требуется явное приведение типов - **многие значения сохранены как тип object вместо float**. По этой причине на текущем этапе нельзя сделать выводы по количественным показателям;
* В таблице представлены коровы **2 разных пород**;
* Все коровы классифицированы по возрасту: **менее 2 лет**, **более 2 лет**;
* Породы пап быков представлены **3 разными видами**;
* Типы пастбищ - **3 разных типа**;
* Названия столбцов не удовлетворяют формату **snake_case** - необходимо выполнить переименование.

`rides`
* **В таблице нет пропущенных значений** - вся информация о поездках и их характеристиках доступна;
* **Среднее время поездки - 17.8 минут, Медианное время - 17.6 минут**. Характеристика **duration** не содержит **явных** выбросов;
* **Среднее расстояние - 3 070 метров, Медианное расстояние 3 133**. Характеристика **distance** не содержит **явных** выбросов;
* Необходимо явного преобразовать столбец **date** - привести к типу datetime;
* Названия столбцов полностью удовлетворяют формату **snake_case**.

`cow_buy`
* Таблица состоит из 20 строк - потенциальных коров для покупки;
* **В таблице нет пропущенных значений** - вся информация о поголовье коров и их характеристиках доступна;
* Требуется явное приведение типов - **многие значения сохранены как тип object вместо float**. По этой причине на текущем этапе нельзя сделать выводы по количественным показателям;
* В таблице представлены коровы **2 разных пород**;
* Все коровы классифицированы по возрасту: **менее 2 лет**, **более 2 лет**;
* Породы пап быков представлены **2 разными видами**;
* Типы пастбищ - **2 разных типа**.
* Названия столбцов не удовлетворяют формату **snake_case** - необходимо выполнить переименование.

